# Mapa de fuentes

Actualizar el mapa con los lugares donde se tomaron las muestras de trabajo de campo (https://dila-ling.github.io/pict-glotales/mapa/). Este mapa tiene la locaclización de fuentes de texto y de audio. Queremos actualizarlo con la localización de las tomas de los nuevos audios incorporados en ago-sep 2023.

- mapa:[../pict-glotales/mapa.md](../pict-glotales/mapa.md)
- datos: [palabras_en_comun_audios](https://docs.google.com/spreadsheets/d/1GtliM_yutOBnWaferOmQjVNLt_xOxz56rst6ZR9MkXc/edit#gid=1308046217), Tanda=2023-08, Comunidad, Latitud, Longitud

También quiero que el mapa tome los datos de un json o de un geojson. Ver [places-by-chapter-map](../notebooks/Del-Barco-Centenera/places-by-chapter-map.ipynb) para construir un [json](https://github.com/hdlabconicet/argentina-y-conquista-del-rio-de-la-plata/blob/7825d3a9f90f4225c322964d5acf4ba067e63b61/assets/js/mapData.js) de datos var places = {'PLACENAME': 'NombreComunidad', 'SOURCES': ['Gerzenstein, 1985', 'Gerzenstein, 1994', 'Gerzenstein, 1999', 'Tacconi, 2015'], 'COLOR': '#4363d8'(según lengua), 'LAT': -25.91667, 'LNG': -57.15}, ...]. [Mapa correspondiente](https://github.com/hdlabconicet/argentina-y-conquista-del-rio-de-la-plata/blob/7825d3a9f90f4225c322964d5acf4ba067e63b61/index.html). [geojson](../argentina-y-conquista-del-rio-de-la-plata/assets/js/mapData.geo.js) y [mapa correspondiente](https://github.com/hdlabconicet/argentina-y-conquista-del-rio-de-la-plata/blob/main/index.html)

Y ver [build-source-map](../notebooks/PICT-20161038/build-source-map.ipynb) para construcción de mapa actual.

In [ ]:
import pandas as pd
from json import dump, load

In [ ]:
sheet = "https://docs.google.com/spreadsheets/d/1GtliM_yutOBnWaferOmQjVNLt_xOxz56rst6ZR9MkXc/edit#gid=1308046217"

sheet = 

df = pd.read_csv(sheet)

df.sample(5)

Hacer un mapa donde los colores de los marcadores indiquen la lengua. En el popup info de nombre de comunidad y fuente